In [1]:
# want to fine tune bert on jnlba dataset
# i use https://huggingface.co/dslim/bert-base-NER for fine tuning. there is also a larger bert model here https://huggingface.co/dslim/bert-large-NER - i want ask matthias about which model to take 

In [1]:
import numpy as np
import pandas as pd

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from datasets import load_dataset

#jnlpba = load_dataset("jnlpba")
#for testing I load only 2% of the dataset 
jnlpba = load_dataset('jnlpba', split=['train', "validation"])
print(jnlpba)

Found cached dataset jnlpba (/Users/maxhager/.cache/huggingface/datasets/jnlpba/jnlpba/1.0.0/3062f220823930cffde7976b694aa67bac3b06c322a02ced92d3761519810ce4)


  0%|          | 0/2 [00:00<?, ?it/s]

[Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 37094
}), Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 7714
})]


In [3]:
# display head of dataset
df = pd.DataFrame(jnlpba[0][0])
df

,id,tokens,ner_tags
0,1,IL-2,1
1,1,gene,2
2,1,expression,0
3,1,and,0
4,1,NF-kappa,9
5,1,B,10
6,1,activation,0
7,1,through,0
8,1,CD28,9
9,1,requires,0


In [4]:
label_list = jnlpba[0].features[f"ner_tags"].feature.names
label_list

['O',
 'B-DNA',
 'I-DNA',
 'B-RNA',
 'I-RNA',
 'B-cell_line',
 'I-cell_line',
 'B-cell_type',
 'I-cell_type',
 'B-protein',
 'I-protein']

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


In [6]:
#example of tokenizing head in several subwords
example = jnlpba[0][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 'IL',
 '-',
 '2',
 'gene',
 'expression',
 'and',
 'N',
 '##F',
 '-',
 'ka',
 '##ppa',
 'B',
 'activation',
 'through',
 'CD',
 '##28',
 'requires',
 'reactive',
 'oxygen',
 'production',
 'by',
 '5',
 '-',
 'lip',
 '##ox',
 '##y',
 '##gen',
 '##ase',
 '.',
 '[SEP]']

In [7]:
#realigning tokens
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [8]:
tokenized_jnlpba = jnlpba[0].map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at /Users/maxhager/.cache/huggingface/datasets/jnlpba/jnlpba/1.0.0/3062f220823930cffde7976b694aa67bac3b06c322a02ced92d3761519810ce4/cache-c38975f88b077ff0.arrow


In [9]:
from transformers import DataCollatorForTokenClassification

#padding sentences dynamically after longest sentence in batch
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

print(data_collator)

DataCollatorForTokenClassification(tokenizer=PreTrainedTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), padding=True, max_length=None, pad_to_multiple_of=None, label_pad_token_id=-100, return_tensors='pt')


In [10]:
#adding metrics
import evaluate

seqeval = evaluate.load("seqeval")

In [11]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [12]:
id2label = {i: label for i, label in enumerate(label_list)}

label2id = {label: i for i, label in enumerate(label_list)}

In [13]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-cased", num_labels=len(label_list), id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [14]:
training_args = TrainingArguments(
    output_dir="jnlpba_model_token_classification",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

#print(len(tokenized_jnlpba[0]))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_jnlpba[0],
    eval_dataset=tokenized_jnlpba[1],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/Users/maxhager/.virtualenvs/thesis/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2
  Number of trainable parameters = 107728139
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

wandb: Currently logged in as: maxhager28 (htw). Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/2 [00:00<?, ?it/s]

KeyError: 5

In [ ]:
trainer.push_to_hub()